<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

Импортируем необходимы модули

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import re
import pymorphy2
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import json

Проверяем, что сколько базовых документов

In [2]:
!ls -lah /data/share/lab05data/base*.txt

-rw-r--r-- 1 bubl bubl 2.9K Oct 25 21:34 /data/share/lab05data/base_10.txt
-rw-r--r-- 1 bubl bubl 3.0K Oct 25 21:34 /data/share/lab05data/base_11.txt
-rw-r--r-- 1 bubl bubl 2.6K Oct 25 21:34 /data/share/lab05data/base_12.txt
-rw-r--r-- 1 bubl bubl 2.6K Oct 25 21:34 /data/share/lab05data/base_13.txt
-rw-r--r-- 1 bubl bubl 9.2K Oct 25 21:34 /data/share/lab05data/base_14.txt
-rw-r--r-- 1 bubl bubl 2.9K Oct 25 21:34 /data/share/lab05data/base_15.txt
-rw-r--r-- 1 bubl bubl 3.6K Oct 25 21:34 /data/share/lab05data/base_16.txt
-rw-r--r-- 1 bubl bubl 3.8K Oct 25 21:34 /data/share/lab05data/base_17.txt
-rw-r--r-- 1 bubl bubl 4.2K Oct 25 21:34 /data/share/lab05data/base_18.txt
-rw-r--r-- 1 bubl bubl 3.7K Oct 25 21:34 /data/share/lab05data/base_19.txt
-rw-r--r-- 1 bubl bubl  803 Oct 25 21:34 /data/share/lab05data/base_1.txt
-rw-r--r-- 1 bubl bubl 3.0K Oct 25 21:34 /data/share/lab05data/base_20.txt
-rw-r--r-- 1 bubl bubl 2.1K Oct 25 21:34 /data/share/lab05data/base_2.txt
-rw-r--r-- 1 bubl bubl 2.0K

Загружаем "базовые" тексты в датафрейм

In [3]:
base_filemask = '/data/share/lab05data/base_{}.txt'
train_df = pd.DataFrame(columns=['id', 'text'])
for i in range(1, 21):
    base_filename = base_filemask.format(i)
    with open(base_filename, 'r') as file:
        train_df = train_df.append({'id': i, 'text': file.readline()}, ignore_index=True)

In [4]:
train_df.head()

,id,text
0,1,<p>Ищем прекрасного программиста 1С в дружный ...
1,2,<p>Предлагаем уникальную возможность присоедин...
2,3,"<p>NetCracker Technology Corp., a large softwa..."
3,4,<p><strong>Senior Java software engineer в Отд...
4,5,"<p>NetCracker Technology Corp., a large softwa..."


Загружаем тексты, по которым необходимо определить сходство с базовыми

In [5]:
test_filemask = '/data/share/lab05data/test_{}.txt'
ids = [2049, 5, 1740, 3593, 3085, 3086, 3600, 1553, 2067, 22, 3607, 536, 25, 538, 3612, 
       2079, 2592, 2084, 1232, 2602, 3627, 1072, 777, 1076, 3125, 56, 570, 572, 1599, 
       3649, 1602, 2628, 1608, 586, 587, 78, 1550, 3665, 85, 89, 1626, 1115, 1131, 1647, 
       2676, 633, 1658, 2174, 1152, 1710, 3715, 2692, 645, 646, 1160, 1674, 3212, 3730, 
       1174, 2200, 3738, 1692, 1695, 1188, 1302, 678, 3239, 174, 1200, 177, 3763, 180, 
       3253, 2230, 2744, 1209, 1723, 701, 1214, 1217, 196, 887, 3272, 3788, 2582, 2255, 
       720, 3795, 1748, 1146, 2265, 1244, 1759, 3808, 3299, 3300, 3818, 3822, 241, 1269, 
       1270, 2688, 1277, 766, 2816, 3842, 262, 1289, 3850, 3340, 3343, 275, 2326, 1305, 
       2332, 1311, 3362, 3292, 808, 298, 2861, 2350, 3375, 2526, 2355, 822, 301, 1849, 
       3387, 1347, 1860, 3398, 3913, 2386, 3925, 2902, 3415, 2618, 869, 2918, 2450, 
       3433, 2411, 3438, 1911, 888, 383, 1920, 2436, 2950, 907, 909, 1423, 1936, 1425, 
       402, 836, 3067, 2457, 496, 2973, 414, 2374, 1441, 2467, 668, 1961, 1963, 943, 
       2377, 2634, 3001, 2492, 3007, 3009, 450, 1991, 2504, 971, 3030, 3545, 3547, 
       1616, 989, 1019, 479, 482, 2812, 307, 2030, 1519, 3052, 498, 2035, 1262, 503, 
       504, 2043, 2044, 3564]

test_df = pd.DataFrame(columns=['id', 'text'])
for i in ids:
    test_filename = test_filemask.format(i)
    with open(test_filename, 'r') as file:
        test_df = test_df.append({'id': i, 'text': file.readline()}, ignore_index=True)

In [6]:
test_df.head()

,id,text
0,2049,<p><strong>Чем придется заниматься:</strong></...
1,5,<p><strong>Обязанности:</strong></p> <ul> <li>...
2,1740,<p><strong>Мы предлагаем всё и сразу: </strong...
3,3593,<p><strong>Специалист по товародвижению в ночн...
4,3085,<p>Компания РосДеньги – надёжная федеральная м...


Используем свой токенизатор. Можно было еще использовать beautifulsoup для удаления тегов, но понял это уже после выполнения заданич

**Да, beautifulsoup помог бы избавиться от тегов и их атрибутов. Например, видно, что в тексты попало много слов "strong" - это тэг, который делает шрифт жирным. Хотя, может быть, он тоже сыграл какую-нибудь положительную роль :)**

**Используется регулярное выражение `([^\w]|[+])`. То есть любой символ, который не является буквой, цифрой или знаком подчеркивания (в нашем случае `\w = [a-zA-Zа-яА-Я_]`) или является знаком плюс (`[+]`) будет рассмотрен в качестве разделителя. Знак плюс в данном случае избыточен, т.к. он уже входит в паттерн `[^\w]`. `t.isspace()` скорей всего тоже не сыграет никакой роли, т.к. все пробельные символы будут исключены, т.к. они будут рассмотрены в качестве разделителя.**

**Раз уж тут используется лемматизатор для русских слов, можно было бы еще использовать лемматизатор для английских слов для симметрии :)**

In [7]:
GROUPING_SPACE_REGEX = re.compile(r'([^\w]|[+])', re.UNICODE)
def simple_word_tokenize(text, _split=GROUPING_SPACE_REGEX.split):
    return [t for t in _split(text.lower()) if t and not t.isspace() and len(t) > 2]

m = pymorphy2.MorphAnalyzer()

def token_r(text):
    words = simple_word_tokenize(text)
    return [m.parse(x)[0].normal_form for x in words]

from nltk.corpus import stopwords
stops_english = set(stopwords.words('english'))
stops_russian = set(stopwords.words('russian'))
stop = stops_english | stops_russian

cv = CountVectorizer(tokenizer=token_r, stop_words=stop, ngram_range=(1,2), binary=True)

In [8]:
GROUPING_SPACE_REGEX.split

<function SRE_Pattern.split(string=None, maxsplit=0, *, source=None)>

Обучаем countvectorizer

In [9]:
cv.fit(train_df.text)

CountVectorizer(analyzer='word', binary=True, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), preprocessor=None,
        stop_words={'didn', 'об', "you're", 'through', 'doesn', 'ни', 'даже', 'уж', 'the', 'один', 'при', 'yourselves', 'which', 'же', 'этом', 'больше', "she's", 'mightn', 'два', 'off', 'did', 'once', 'our', 'но', "doesn't", 'такой', 'were', 'hasn', 'без', 'most', 'себя', 'потом', 'these', 'but', 'можно', '..., 'сам', 'над', 'какой', 'уже', 'about', 'whom', 'была', 'мне', 'should', 'тут', 'о', 'все', 'then'},
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=<function token_r at 0x7f7fbbad29d8>, vocabulary=None)

Сотрим, что за фичи получились

In [10]:
cv.get_feature_names()[:10]

['100',
 '100 таблица',
 '100 это',
 '300',
 '300 строка',
 'abilities',
 'abilities business',
 'ability',
 'ability interact',
 'ability travel']

Векторизуем наши "базовые" документы

**До этого CountVectorizer уже был обучен, тут можно было применить только cv.transform. А то получается, что мы заново тут его обучаем**

In [11]:
%%time
base_matrix = cv.fit_transform(train_df.text).toarray()

CPU times: user 715 ms, sys: 0 ns, total: 715 ms
Wall time: 716 ms


Смотрим топ слов

**Вот оно топ слово из тэга `<strong>` :)**

In [12]:
sum_words = base_matrix.sum(axis=0)
words_freq = [(word, sum_words[idx]) for word, idx in cv.vocabulary_.items()]
words_freq = sorted(words_freq, key = lambda x: x[1], reverse=True)
words_freq[:20]

[('strong', 19),
 ('работа', 17),
 ('команда', 16),
 ('разработка', 16),
 ('знание', 16),
 ('опыт', 16),
 ('компания', 15),
 ('разработчик', 14),
 ('требование', 14),
 ('опыт разработка', 14),
 ('опыт работа', 13),
 ('strong strong', 13),
 ('решение', 13),
 ('результат', 13),
 ('проект', 12),
 ('свой', 12),
 ('strong требование', 12),
 ('наш', 11),
 ('продукт', 11),
 ('понимание', 11)]

Векторизуем "тестовые" документы

In [13]:
test_matrix = cv.transform(test_df.text).toarray()

Находим косинусное сходство между "тестовыми" и "базовыми" документами

In [14]:
cos_m = cosine_similarity(base_matrix, test_matrix)
cos_m

array([[0.07906198, 0.08959404, 0.04702304, ..., 0.09153773, 0.01280684,
        0.02036157],
       [0.18943817, 0.12281814, 0.09100315, ..., 0.20667725, 0.22306433,
        0.15762208],
       [0.01133695, 0.01124128, 0.01179987, ..., 0.02871288, 0.01928234,
        0.01021899],
       ...,
       [0.21540966, 0.1898594 , 0.09964688, ..., 0.15518264, 0.18997346,
        0.16396378],
       [0.16642306, 0.22690061, 0.10826163, ..., 0.17386733, 0.2063972 ,
        0.19689038],
       [0.17250998, 0.22623294, 0.09267307, ..., 0.15785235, 0.25870726,
        0.19061093]])

Суммируем косинусную близость по каждому тестовому документу и считаем среднее

In [15]:
cos_m_sum = np.sum(cos_m, axis=0)
test_df['def'] = cos_m_sum > cos_m_sum.mean()

Смотрим, что же получилось

In [16]:
test_df.head()

,id,text,def
0,2049,<p><strong>Чем придется заниматься:</strong></...,True
1,5,<p><strong>Обязанности:</strong></p> <ul> <li>...,True
2,1740,<p><strong>Мы предлагаем всё и сразу: </strong...,False
3,3593,<p><strong>Специалист по товародвижению в ночн...,False
4,3085,<p>Компания РосДеньги – надёжная федеральная м...,True


In [17]:
test_df['def'].value_counts()

False    101
True      99
Name: def, dtype: int64

Формируем ответ для чекера

In [18]:
answer = {
    'defined': test_df[test_df['def']].id.tolist(),
    'other': test_df[~test_df['def']].id.tolist()
}

In [23]:
with open('/data/home/alexander.alexandrov/lab05.json', 'w') as outfile:
    json.dump(answer, outfile)